In [1]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pyopencl as cl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import signal
from matplotlib import rcParams
from scipy import io
import os

import locale
os.environ["PYTHONIOENCODING"] = "utf-8"
# scriptLocale=locale.setlocale(category=locale.LC_ALL, locale="en_GB.UTF-8")

In [2]:
# Widening the screen
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

rcParams['figure.dpi'] = 1200
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'
rcParams["mathtext.fontset"] = 'stix'
plt.rc('font', size=16)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=16)   # fontsize of the figure title

# if not os.path.exists("data"):
#     os.makedirs("data")

## Model description

equations:

$\displaystyle \frac{\partial u}{\partial t}=  -k_d u +  f(u,v)\cdot v -\rho g(u,v) \cdot u +D_u \Delta u$

$\displaystyle \frac{\partial v}{\partial t}=+k_d u- f(u,v)\cdot v -\rho g(u,v) \cdot u +D_v \Delta v $

$u$: DNA-bound proteins concentration

$v$: free protein concentration

$\rho$: DNA length or stability of DNA-bound protein


$D_u<<D_v$

Here,we take $\displaystyle f(u,v)= k_1 u +\frac{\tilde{b}}{k_3}=k_1u+b$ , $\displaystyle g(u,v)=\frac{ k_2  \cdot \beta}{k_2+u+v}$

In [3]:
length    = 1024       #512.0      Length of the physical landscape
n         = 2048       # 2048      Size of the 2D grid

#n=2048
endtime   = 10000.0     #1000       end time
dT        = 0.005      # 0.02      calculate time step
nplot     = 100
nsteps    = np.ceil(endtime/nplot/dT).astype(int)    #number of time steps

dX = 0.20    #0.2  length/n      # Spatial step size
dY = 0.20    #0.2  length/n      # Spatial step size
Grid_Width  = n
Grid_Height = n
# %% Reporting in the simulation on the console
print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

 Current grid dimensions: 2048 x 2048 cells



## simulation settings

In [4]:
# Setting up the OpenCL context
DeviceNr = 0   # 0 = GTX 960M
platform = cl.get_platforms()
Devices  = platform[0].get_devices()  # 0 = GPU
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)
mf       = cl.mem_flags # Memory flags are set
print(" Compute Device: %s\n" % Devices[DeviceNr].name)

 Compute Device: Apple M2 Ultra



## Parameters in the model


In [5]:
kd=0.36            # Dissociation rate constant
k3=6.7             # intensity conversion factor
k1=1.35            #recruitment rate coefficient
tildeb=60          #self-triggering rate
b=tildeb/k3
rho=5.0         # stability: default set as estimated half-saturation concentration of DNA
k2=2.0          # half-saturation constant : threshold concentration
beta=17.8*1.1   # maximal reduction of intensity per hour (estimated)
Du=0.17*0.05    # diffusion for DNA-bound protein
Dv=0.17         # diffusion for free protein
cv=0.016*6.7    #constant speed up calculation

In [6]:

FpMax=80.0 # total protein concentration 
FpMin=10.0
rhoMin=1.0 # stability of conbined DNA-protein
rhoMax=22

In [7]:
## The array is allocated on the GPU and the initial values are copied onto it
## set the free protein series
T        = np.tile(np.logspace(np.log10(FpMin),np.log10(FpMax),n),n) # total protein X-axis
seed     = (np.random.rand(n*n)-0.5)*2.0
v        = T+ (seed-seed.mean())*0.5

v_host   = v.astype(np.float32)
v_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=v_host)


## set the DNA-bound protein concentration
u0= 0.5 # initial DNA-bound protein concentration
u= u0+  (np.random.rand(n*n)-0.5)*0.05   # u ~(0,0.5)

u_host   = u.astype(np.float32)
u_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=u_host)

## set the rho series 
rho_arr=np.logspace(np.log10(rhoMax),np.log10(rhoMin),n).repeat(n)
#rho_arr=np.linspace(rhoMax,rhoMin,n).repeat(n)

rho_arr_host   = rho_arr.astype(np.float32)
rho_arr_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=rho_arr_host)

# cl.enqueue_copy(queue, rho_arr_host, rho_arr_g)

v1=v.reshape(n,n)
rho_arr1=rho_arr.reshape(n,n)

if not os.path.exists("Data"):
    os.makedirs("Data")
io.savemat('Data/initialv.mat', {'v': v1})
io.savemat('Data/initialrho.mat', {'rho': rho_arr1})

In [8]:
import numpy as np

def log_scale_points(ymin, ymax, n, special_values=None):
    # 检查输入有效性
    if ymin <= 0 or ymax <= 0:
        raise ValueError("ymin 和 ymax 必须大于 0 才能使用对数坐标。")
    if ymin >= ymax:
        raise ValueError("ymin 必须小于 ymax。")
    if n < 2:
        raise ValueError("n 必须大于或等于 2。")

    # 在对数空间中生成均匀间隔的点
    log_ymin = np.log10(ymin)
    log_ymax = np.log10(ymax)
    log_points = np.linspace(log_ymin, log_ymax, n)

    # 将对数空间的点映射回线性空间
    points = 10 ** log_points

    # 如果有特殊数值，添加这些值并排序
    if special_values is not None:
        points = np.concatenate((points, special_values))
        points = np.unique(points)
        points.sort()
    
    return points

def get_positions(ymin, ymax, values, n):
    # 检查输入有效性
    if ymin <= 0 or ymax <= 0:
        raise ValueError("ymin 和 ymax 必须大于 0 才能使用对数坐标。")
    if ymin >= ymax:
        raise ValueError("ymin 必须小于 ymax。")

    # 将输入值转换为对应的对数空间位置
    log_ymin = np.log10(ymin)
    log_ymax = np.log10(ymax)
    log_values = np.log10(values)

    # 计算每个值在对数坐标下的位置（归一化到 1 到 N 之间）
    positions = 1 + (log_values - log_ymin) / (log_ymax - log_ymin) * (n - 1)
    
    return positions

In [9]:
with open('SpatialFunctions_iPy.cl', 'r',encoding='utf-8') as myfile:
   SpatialFunctions = myfile.read()

In [10]:
# List of parameters for the OpenCL kernel. Seperate with comma without spaces
# Setting up the parameters for the Kernel
PassVars="kd,k3,k1,b,k2,rho,cv,beta,Du,Dv,dX,dY,dT,Grid_Width,Grid_Height"
PassVals=eval(PassVars)
PassVars=PassVars.split(',')
Params=""

for ii in range(len(PassVals)):
    Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"

In [11]:
ComputeCode = """

///////////////////////////////////////////////////////////////////////////////
// Simulation kernel, __global float* rho_arr
///////////////////////////////////////////////////////////////////////////////   

__kernel void SimulationKernel (__global float* u, __global float* v, __global float* rho_arr)
{
    size_t current  = get_global_id(0);
    size_t row      = floor((float)current/(float)Grid_Width);
    size_t column   = current%Grid_Width;

        if (row > 0 && row < Grid_Width-1 && column > 0 && column < Grid_Height-1)
            {
                
             float dudt =  cv*(-kd*u[current] + (k1*u[current]+b)*v[current] - rho_arr[current]*k2*beta*u[current]/(k2+u[current]+v[current])) + Du*d2_dxy2(u); 
             float dvdt = -cv*(-kd*u[current] + (k1*u[current]+b)*v[current] - rho_arr[current]*k2*beta*u[current]/(k2+u[current]+v[current])) + Dv*d2_dxy2(v); 
              
             u[current] = u[current]+dudt*dT;
             v[current] = v[current] +dvdt*dT; 
            } 
        // HANDLE Boundaries
        else 
            {
             
             NeumannBoundaries(u);
             NeumannBoundaries(v);
             
            }
} // End SimulationKernel
"""

In [12]:
program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

from ipywidgets import FloatProgress
from IPython.display import display

# Setting up a progress bar for the simulation
print("Progress :");
PB = FloatProgress(min=0, max=nplot); display(PB) 
Us=np.zeros((Grid_Width, Grid_Height, nplot))
Vs=np.zeros((Grid_Width, Grid_Height, nplot))

# Set up simulation parameters
global_size = u_host.shape

# Start the timer:
start_time = time.time()

# Starting the loop
for ii in range(1,nplot):
    # The simulation
    for jj in range(1,nsteps):    
        
#         cl.enqueue_copy(queue, A_host, A_g)
#         phic = signal.convolve2d(A_host.reshape(n,n), C, mode='same', boundary='wrap')                     
#         phic_host = phic.reshape(n*n,order='C').astype(np.float32)
#         phic_g    = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=phic_host)
#         program.SimulationKernel(queue, global_size, None, A_g, B_g, phic_g,rho_arr_g)
        program.SimulationKernel(queue, global_size, None, u_g, v_g,rho_arr_g)

    # Get the data from the GPU
    cl.enqueue_copy(queue, u_host, u_g)
    cl.enqueue_copy(queue, v_host, v_g)
    
    # We store the state of the system for <NumPlot> different times.
    Us[:,:,ii] = u_host.reshape(Grid_Width, Grid_Height)
    Vs[:,:,ii] = v_host.reshape(Grid_Width, Grid_Height)
    
    PB.value += 1 # signal to increment the progress bar

# Determining the time that we used for the simulation
elapsed_time = time.time() - start_time    
print(" Simulation took      : %1.1f (s)" % (elapsed_time))


Progress :


FloatProgress(value=0.0)

AGX: exceeded compiled variants footprint limit


 Simulation took      : 581.9 (s)


In [13]:
### The Simulation loop

nbin=4
# viridis, Reds, Greens, YlGn
# plot_color_gradients('Sequential',
#                      ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
#                       'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
#                       'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'])
# plot_color_gradients('Sequential (2)',
#                      ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
#                       'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
#                       'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper'])

#orig_map=plt.cm.get_cmap('viridis') # viridis  YlGn, summer
orig_map=plt.colormaps['GnBu']
reversed_map = orig_map.reversed()

from matplotlib import ticker
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
im0 = ax[0].imshow(np.log(u_host.reshape(n,n)),interpolation='nearest', cmap=reversed_map,
                   extent=[0,length,0,length],clim=(0,4));#, clim=(0,0.1));
ax[0].set_title('$n$ density ($\mu$M/$\mu$m$^2$)');
cbar=plt.colorbar(im0, ax=ax[0],fraction=0.046,pad=0.04);
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
# Xlabels = np.linspace(BioMin,BioMax,nbin).round(1)
Xlabels = np.logspace(np.log10(FpMin),np.log10(FpMax),nbin).round(0)
Ylabels = np.logspace(np.log10(rhoMin),np.log10(rhoMax),nbin).round(1)
ax[0].set_xticks(np.linspace(0,length,nbin))
ax[0].set_xticklabels(Xlabels)
ax[0].set_yticks(np.linspace(0,length,nbin))
ax[0].set_yticklabels(Ylabels)
ax[0].set_xlabel('Initial free protein concentration , $u$');
ax[0].set_ylabel(' combined effect of DNA, $\\rho$');

im1 = ax[1].imshow(np.log(v_host.reshape(n,n)),interpolation='nearest',cmap=reversed_map,
                  extent=[0,length,0,length],clim=(0,2));#, clim=(0,0.5));

ax[1].set_title('$v$ DNA-bound protein density ($\mu$M/$\mu$m$^2$)');
cbar=plt.colorbar(im1, ax=ax[1],fraction=0.046,pad=0.04);
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
# Xlabels = np.linspace(BioMin,BioMax,nbin).round(0)
Xlabels = np.logspace(np.log10(FpMin),np.log10(FpMax),nbin).round(0)
#Ylabels = rho*np.logspace(np.log10(rhoMin),np.log10(rhoMax),nbin).round(1)
Ylabels = np.logspace(np.log10(rhoMin),np.log10(rhoMax),nbin).round(1)

ax[1].set_xticks(np.linspace(0,length,nbin))
ax[1].set_xticklabels(Xlabels)
ax[1].set_yticks(np.linspace(0,length,nbin))
ax[1].set_yticklabels(Ylabels)
ax[1].set_xlabel('free protein concentration, $\\tau$');
# ax[1].set_ylabel('diffusion ratio, $\delta$');

fig.tight_layout()

text=fig.suptitle("Time: %1.0f of %1.0f" % (endtime, endtime),x=0.5, y=0.05, fontsize=16);
#fig.savefig('Fig5A_PhaseSeparation_%03g'% 10+'.pdf',bbox_inches='tight',dpi=600)

In [ ]:
### The Simulation loop
nbin=4

# viridis, Reds, Greens, YlGn
# plot_color_gradients('Sequential',
#                      ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
#                       'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
#                       'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'])
# plot_color_gradients('Sequential (2)',
#                      ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
#                       'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
#                       'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper'])

#orig_map=plt.cm.get_cmap('viridis') # viridis  YlGn, summer
orig_map=plt.colormaps['GnBu']
reversed_map = orig_map.reversed()

from matplotlib import ticker
fig, ax2 = plt.subplots(1, 1, figsize=(10, 5))
# im0 = ax[0].imshow(np.log(A_host.reshape(n,n))[::-1, :],interpolation='nearest', cmap=reversed_map,
#                     extent=[0,length,0,length],clim=(0,5));#, clim=(0,0.1));
im0 = ax2.imshow(np.log(u_host.reshape(n,n)),interpolation='nearest', cmap=reversed_map,
                   extent=[0,length,0,length],clim=(0,5));#, clim=(0,0.1));


# 将标题移到colorbar右侧
#ax2.set_title('DNA-bound protein concentration', loc='right')
#ax2.set_title('DNA-bound protein concentration ');
cbar=plt.colorbar(im0, ax=ax2,fraction=0.046,pad=0.04);
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()

#cbar.ax.yaxis.set_tick_params(direction='in')
Xlabels = np.logspace(np.log10(FpMin),np.log10(FpMax),nbin).round(0)
Ylabels = np.logspace(np.log10(rhoMin),np.log10(rhoMax),nbin).round(1)
#Ylabels = np.linspace(rhoMin,rhoMax,nbin).round(1) # if linespace for y-axis

special_values = [1.0, 2.0, 8.0, 22.0]
Ylabels = special_values
positions = get_positions(rhoMin, rhoMax, Ylabels, length)
#plot for marker at y-axis for 'rho=4' 
target_values = [4]
Y_Markerline = get_positions(rhoMin, rhoMax, target_values, length)
# Y_Markerline = length*(np.log10(4.0)/(np.log10(rhoMax)-np.log10(rhoMin)))

ax2.set_xticks(np.linspace(0,length,nbin))
ax2.set_xticklabels(Xlabels)
ax2.set_yticks(np.linspace(0,length,nbin))
ax2.set_yticklabels(Ylabels)

ax2.plot([1, length], [Y_Markerline, Y_Markerline], '--', lw=1,color='gray', alpha=0.2)
# 将ticks朝内# 调整x轴和y轴ticks数字之间的间距
ax2.tick_params(axis='x', pad=8)
ax2.tick_params(axis='y', pad=8)
ax2.yaxis.set_ticks_position('both')
ax2.xaxis.set_ticks_position('both')
ax2.tick_params(axis='both', direction='in')
ax2.set_xlabel('Initial free protein concentration');
ax2.set_ylabel('Stability of DNA-bound protein, $\\rho$');


In [ ]:
fig.savefig('Fig2c_phase.pdf',bbox_inches='tight',dpi=600)
fig.savefig('Fig2c_phase.jpg',bbox_inches='tight',dpi=600)   